<a href="https://colab.research.google.com/github/stefkong1982/netology.ru/blob/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/Kondratev_Itog_Analitika_bolshih_dannyh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Итоговая работа "Аналитика больших данных"

**Преподаватель:** Алексей Кузьмин

В данном разделе мы выступим в роли data scientist и попытаемся построить простую модель для рекомендации фильмов пользователям.

**1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы**



Сформировав общий топ фильмов в прошлой практике, мы хотим сделать шаг вперед и начать советовать пользователю те фильмы, которые могли бы быть для него наиболее интересны. Наша цель - научится предсказывать оценку фильма пользователем. Для тестирования модели найдем пользователя, который поставил больше всего оценок

**2. Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок**



Отберем фильмы, которые оценил данный пользователь

**3. Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь**



Для построения модели нам нужны признаки. В качестве таковых будем использовать:

* Год выхода

* Жанры
* Общее количество оценок

* Суммарную оценку




**4. Добавьте к датафрейму из задания 3 столбцы:**

  * По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет

  * столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей



Теперь все готово и можно строить модель


**5. Сформируйте X_train, X_test, y_train, y_test**

**6. Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах**

**7. Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии**


Вторая часть практики Python связана со Spark’ом

**8. Загрузить данные в spark**

**9. Средствами спарка вывести среднюю оценку для каждого фильма**

**10. Посчитайте средствами спарка среднюю оценку для каждого жанра**

**11. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)**


## Решение

В данном разделе мы выступим в роли data scientist и попытаемся построить простую модель для рекомендации фильмов пользователям.

### 1 - Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы

Сформировав общий топ фильмов в прошлой практике, мы хотим сделать шаг вперед и начать советовать пользователю те фильмы, которые могли бы быть для него наиболее интересны. Наша цель - научится предсказывать оценку фильма пользователем. Для тестирования модели найдем пользователя, который поставил больше всего оценок

In [260]:
import pandas as pd

# Ссылка на файл с оценками
ratings_url = "https://github.com/stefkong1982/netology.ru/raw/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/u.data.csv"

# Загружаем файл с оценками в датафрейм
ratings = pd.read_csv(ratings_url, sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Ссылка на файл с фильмами
movies_url = "https://github.com/stefkong1982/netology.ru/raw/Master/Analitika_bolshih_dannyh/Itogovaya_rabota_Analitika_bolshih_dannyh/u.item.csv"

# Загружаем файл с фильмами в датафрейм
movies = pd.read_csv(movies_url, sep='|', encoding='latin-1', header=None,
                     names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL',
                            'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
                            'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                            'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])

In [261]:
# Посмотрим на первые строки каждого датафрейма
ratings.head(3)

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116


```
u.data — полный набор данных u, 100 000 оценок 943 пользователей по 1682 элементам.
```
```
Каждый пользователь оценил не менее 20 фильмов. Пользователи и элементы
пронумерованы последовательно с 1. Данные случайным образом
заказал.
```

```
Столбцы:

user_id	                          item_id	      rating	      timestamp
идентификатор пользователя | идентификатор элемента | рейтинг | временная метка
```
```
Это список с разделителем табуляция sep='\t'
```
```
Временные метки представляют собой unix-секунды с 01.01.1970 UTC.
```

In [262]:
movies.head(3)

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


```
u.item - Информация фильмах:
```

```
Столбцы:

идентификатор фильма | название фильма | дата выпуска | дата выхода видео |
URL-адрес IMDb | неизвестный | Действие | Приключения | Анимация |
Детская | комедия | Преступность | Документальный | Драма | Фэнтези |
Фильм-Нуар | Ужасы | Музыкальный | Тайна | Романтика | Научная фантастика |
Триллер | Война | Западная |
```           
```
Это список с разделителем sep='|'
```
```
Последние 19 полей — жанры,

1 — фильм. относится к этому жанру

0 - означает, что это не так

фильмы могут быть внесколько жанров одновременно
```
```
Идентификаторы фильмов (movie_id) — это те, которые используются в наборе данных u.data.
```

### 2 - Средствами Pandas, используя dataframe `ratings`, найдите id пользователя, поставившего больше всего оценок

In [263]:
# Группируем данные по 'user_id' и считаем количество оценок каждого пользователя
# По умолчанию метод `value_counts()` в Pandas сортирует уникальные значения в порядке убывания количества их появлений.
user_ratings_count = ratings['user_id'].value_counts()

# Определяем top_user_id - ID пользователя с наибольшим количеством оценок
top_user_id = user_ratings_count.index[0]

print(f"ID пользователя с наибольшим количеством оценок: {top_user_id}")
print(f"Количество оценок пользователя: {user_ratings_count.values[0]}")


ID пользователя с наибольшим количеством оценок: 405
Количество оценок пользователя: 737


Отберем фильмы, которые оценил данный пользователь

### 3 - Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь

Для построения модели нам нужны признаки. В качестве таковых будем использовать:
* Год выхода
* Жанры
* Общее количество оценок
* Суммарную оценку


In [264]:
# Вычисляем общее количество оценок и суммарную оценку от всех пользователей для каждого фильма
ratings['total_ratings_all'] = ratings.groupby('movie_id')['rating'].transform('count')
ratings['sum_ratings_all'] = ratings.groupby('movie_id')['rating'].transform('sum')

# Отображаем обновленный датафрейм ratings
ratings

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all
0,196,242,3,881250949,117,467
1,186,302,3,891717742,297,1236
2,22,377,1,878887116,13,28
3,244,51,2,880606923,81,280
4,166,346,1,886397596,126,459
...,...,...,...,...,...,...
99995,880,476,3,880175444,160,483
99996,716,204,5,879795543,350,1342
99997,276,1090,1,874795795,37,89
99998,13,225,2,882399156,109,317


In [265]:
# Оставляем только фильмы, оцененные пользователем - 405 (top_user_id)
user_ratings = ratings[ratings['user_id'] == top_user_id]

In [266]:
user_ratings

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all
12276,405,56,4,885544911,394,1600
12383,405,592,1,885548670,9,30
12430,405,1582,1,885548670,1,1
12449,405,171,1,885549544,65,252
12460,405,580,1,885547447,32,108
...,...,...,...,...,...,...
98956,405,375,1,885546835,23,45
98978,405,445,4,885548435,22,77
99148,405,1246,1,885547735,7,11
99465,405,196,1,885546112,251,983


### 4 - Добавьте к датафрейму из задания 3 столбцы:

* Жанр
* Год выхода
* столбец с общим количеством оценок от всех пользователей на фильм - уже добавили на в 3-м задании
* столбец и суммарной оценкой от всех пользователей - уже добавили на в 3-м задании


In [267]:
# Добавляем к датафрейму столбцы 'жанр', 'год выхода'
user_ratings = user_ratings.merge(movies[['movie_id', 'release_date'] + list(movies.columns[5:24])], on='movie_id', how='left')

In [268]:
user_ratings

,user_id,movie_id,rating,timestamp,total_ratings_all,sum_ratings_all,release_date,unknown,Action,Adventure,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,885544911,394,1600,01-Jan-1994,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,405,592,1,885548670,9,30,01-Jan-1995,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,405,1582,1,885548670,1,1,01-Jan-1947,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,405,171,1,885549544,65,252,01-Jan-1991,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,405,580,1,885547447,32,108,01-Jan-1995,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,405,375,1,885546835,23,45,01-Jan-1995,0,0,0,...,0,0,0,0,0,0,0,0,0,0
733,405,445,4,885548435,22,77,01-Jan-1945,0,0,0,...,0,0,1,0,0,0,0,0,0,0
734,405,1246,1,885547735,7,11,01-Jan-1995,0,0,0,...,0,0,0,0,0,0,0,0,0,0
735,405,196,1,885546112,251,983,01-Jan-1989,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Теперь все готово и можно строить модель

### 5 - Сформируйте X_train, X_test, y_train, y_test

In [271]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

Среднеквадратичная ошибка: 1.8697758583200908e+16


In [272]:
# Выделение признаков (X) и целевой переменной (y)
X = user_ratings[list(user_ratings.columns[4:26])]
y = user_ratings['rating']

In [273]:
X.head(1)

,total_ratings_all,sum_ratings_all,release_date,unknown,Action,Adventure,Animation,Children,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,394,1600,01-Jan-1994,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [274]:
# Применение One-Hot Encoding к категориальным признакам
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(X[['release_date']])
X_encoded_df = pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names_out(['release_date']))
X = pd.concat([X.drop(['release_date'], axis=1), X_encoded_df], axis=1)

In [275]:
# Разбиение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 6 - Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

In [278]:
# Создание и обучение модели регрессии
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [277]:
X_train

,total_ratings_all,sum_ratings_all,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,release_date_23-May-1997,release_date_24-Sep-1996,release_date_25-Dec-1997,release_date_25-Oct-1996,release_date_25-Sep-1995,release_date_26-Dec-1997,release_date_27-Jun-1997,release_date_29-Mar-1988,release_date_29-Mar-1996,release_date_30-Jan-1998
135,2,6,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
545,39,110,0,0,1,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,29,70,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
340,40,108,0,1,1,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,1,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,102,375,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106,39,112,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
270,13,42,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
435,5,12,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 7 - Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [279]:
# Предсказание на тестовом наборе
y_pred = model.predict(X_test)

In [280]:
y_pred

array([1.26154804e+00, 4.33190405e+00, 1.17039251e+00, 1.39033341e+00,
       3.13583302e+00, 3.45345819e+00, 1.05913520e+00, 1.88615382e+00,
       5.45690739e+00, 1.99810076e+00, 4.86439466e+00, 2.14037931e+00,
       1.47469795e+00, 2.87104106e+00, 9.82875466e-01, 1.30025160e+00,
       1.52032769e+00, 2.59701121e+00, 5.89109142e+08, 1.30461419e+00,
       1.93609893e+00, 2.75143683e+00, 1.22639680e+00, 1.97826934e+00,
       1.42036140e+00, 1.16818058e+00, 1.44793856e+00, 1.78773344e+00,
       1.42143071e+00, 2.75837243e+00, 1.05020905e+00, 3.41797400e+00,
       1.94789863e+00, 3.37297678e+00, 1.26947808e+00, 1.27596927e+00,
       1.36920965e+00, 2.80258405e+00, 2.71675348e-01, 1.32027495e+00,
       2.95918965e+00, 5.72487950e-01, 1.31053066e+00, 5.87970109e+08,
       2.85520685e+00, 2.39612961e+00, 1.47058928e+00, 5.26312542e+00,
       1.28129828e+00, 4.28641713e+00, 2.85098660e+00, 5.88881075e+08,
       1.33078957e+00, 4.38672662e+00, 1.43131769e+00, 1.60979474e+00,
      

In [281]:
# Оценка качества модели
mse = mean_squared_error(y_test, y_pred)
print(f"Среднеквадратичная ошибка: {mse}")

Среднеквадратичная ошибка: 1.8697758583200908e+16
